In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import pandas as pd

class TextClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(TextClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)  
        self.relu = nn.ReLU() 
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x  

data = pd.read_csv('/Users/basil/Downloads/ml_project.csv')
data = data.dropna(subset=['content', 'gold_label'])

X_text = data['content'].tolist()
y = data['gold_label'].tolist()

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(X_text).toarray()

classes = sorted(list(set(y)))  
class_to_index = {cls: idx for idx, cls in enumerate(classes)}
index_to_class = {idx: cls for cls, idx in class_to_index.items()}
y = [class_to_index[label] for label in y]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)

input_size = X_train.shape[1]
hidden_size = 128
output_size = len(classes)
learning_rate = 0.001
epochs = 20
batch_size = 64 

model = TextClassifier(input_size, hidden_size, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)


from torch.utils.data import DataLoader, TensorDataset

train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Training loop
for epoch in range(1, epochs + 1):
    model.train()
    epoch_loss = 0
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    avg_loss = epoch_loss / len(train_loader)
    if epoch % 5 == 0 or epoch == 1:
        print(f"Epoch [{epoch}/{epochs}], Loss: {avg_loss:.4f}")

model.eval()
with torch.no_grad():
    outputs = model(X_test)
    _, predicted = torch.max(outputs, 1)

y_pred = [index_to_class[pred.item()] for pred in predicted]
y_true = [index_to_class[true.item()] for true in y_test]

accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='weighted', zero_division=0)
recall = recall_score(y_true, y_pred, average='weighted', zero_division=0)
f1 = f1_score(y_true, y_pred, average='weighted', zero_division=0)
conf_matrix = confusion_matrix(y_true, y_pred)

print(f"\nAccuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-Score: {f1:.2f}")
print("Confusion Matrix:")
print(conf_matrix)

print("\nClassification Report:")
print(classification_report(y_true, y_pred, zero_division=0))


Epoch [1/20], Loss: 0.6643
Epoch [5/20], Loss: 0.0093
Epoch [10/20], Loss: 0.0024
Epoch [15/20], Loss: 0.0010
Epoch [20/20], Loss: 0.0005

Accuracy: 97.58%
Precision: 0.98
Recall: 0.98
F1-Score: 0.98
Confusion Matrix:
[[66  0  0  1  2]
 [ 0 60  0  0  0]
 [ 0  0 43  0  1]
 [ 0  0  0 57  1]
 [ 1  0  1  0 56]]

Classification Report:
                    precision    recall  f1-score   support

          business       0.99      0.96      0.97        69
     entertainment       1.00      1.00      1.00        60
science-technology       0.98      0.98      0.98        44
            sports       0.98      0.98      0.98        58
             world       0.93      0.97      0.95        58

          accuracy                           0.98       289
         macro avg       0.98      0.98      0.98       289
      weighted avg       0.98      0.98      0.98       289

